In [1]:
import pandas as pd
import numpy as np
import shap

In [2]:
df = pd.read_excel("Completed Orders Safex.xls")

In [3]:
df

,TradeDate,Instrument,CallPut,Strike,No of Contracts Trade,Option Premium,Origin,Underlying's Future Level,ExpiryDate,Volatility
0,2009-07-31 12:02:58,ALSI,C,26000.0,250,3831.00,F,22114.00,2010-03-18,21.67
1,2009-07-31 12:29:19,ALSI,C,24000.0,1500,8229.00,P,22080.00,2010-03-18,22.01
2,2009-07-31 12:40:48,ALSI,C,24000.0,1500,8209.00,Q,22080.00,2010-03-18,21.98
3,2009-07-31 12:44:17,ALSI,P,18000.0,1000,5012.00,P,22150.00,2010-03-18,30.00
4,2009-07-31 12:44:18,ALSI,C,25000.0,1000,5523.00,P,22150.00,2010-03-18,21.30
...,...,...,...,...,...,...,...,...,...,...
53477,2012-09-05 00:00:00,SOLQ,P,340.0,30,1297.00,L,351.62,2012-12-20,24.49
53478,2012-09-05 00:00:00,SOLQ,P,340.0,35,1070.72,L,351.62,2012-12-20,21.32
53479,2012-09-05 00:00:00,SOLQ,P,340.0,35,1106.70,L,351.62,2012-12-20,21.83
53480,2012-09-05 00:00:00,SOLQ,P,340.0,50,1070.72,L,351.62,2012-12-20,21.32


In [4]:
df.drop(['Origin','No of Contracts Trade'],axis=1,inplace=True)
df['Tenor'] = (df['ExpiryDate']-df['TradeDate']).apply(lambda x : x.days)
df.drop(['ExpiryDate','TradeDate'],axis=1,inplace=True)

In [5]:
df

,Instrument,CallPut,Strike,Option Premium,Underlying's Future Level,Volatility,Tenor
0,ALSI,C,26000.0,3831.00,22114.00,21.67,229
1,ALSI,C,24000.0,8229.00,22080.00,22.01,229
2,ALSI,C,24000.0,8209.00,22080.00,21.98,229
3,ALSI,P,18000.0,5012.00,22150.00,30.00,229
4,ALSI,C,25000.0,5523.00,22150.00,21.30,229
...,...,...,...,...,...,...,...
53477,SOLQ,P,340.0,1297.00,351.62,24.49,106
53478,SOLQ,P,340.0,1070.72,351.62,21.32,106
53479,SOLQ,P,340.0,1106.70,351.62,21.83,106
53480,SOLQ,P,340.0,1070.72,351.62,21.32,106


In [6]:
df = df[df['Instrument']=='ALSI']

In [7]:
df.drop(['Instrument'],axis=1,inplace=True)
df['Put_Flag'] = pd.get_dummies(df['CallPut'],drop_first=True)

C:\Users\alanw\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\alanw\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
df

,CallPut,Strike,Option Premium,Underlying's Future Level,Volatility,Tenor,Put_Flag
0,C,26000.0,3831.00,22114.0,21.67,229,0
1,C,24000.0,8229.00,22080.0,22.01,229,0
2,C,24000.0,8209.00,22080.0,21.98,229,0
3,P,18000.0,5012.00,22150.0,30.00,229,1
4,C,25000.0,5523.00,22150.0,21.30,229,0
...,...,...,...,...,...,...,...
53451,P,31000.0,10976.00,31300.0,18.55,106,1
53452,P,28000.0,3408.00,31300.0,22.33,106,1
53453,P,28000.0,3292.00,31090.0,21.22,106,1
53454,P,22000.0,3563.66,31232.0,31.25,288,1


In [11]:
df['Volatility'] = df['Volatility'].apply(lambda x : x/100)
df['Tenor'] = df['Tenor'].apply(lambda x : x/365)

C:\Users\alanw\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\alanw\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df

,CallPut,Strike,Option Premium,Underlying's Future Level,Volatility,Tenor,Put_Flag
0,C,26000.0,3831.00,22114.0,0.2167,0.627397,0
1,C,24000.0,8229.00,22080.0,0.2201,0.627397,0
2,C,24000.0,8209.00,22080.0,0.2198,0.627397,0
3,P,18000.0,5012.00,22150.0,0.3000,0.627397,1
4,C,25000.0,5523.00,22150.0,0.2130,0.627397,0
...,...,...,...,...,...,...,...
53451,P,31000.0,10976.00,31300.0,0.1855,0.290411,1
53452,P,28000.0,3408.00,31300.0,0.2233,0.290411,1
53453,P,28000.0,3292.00,31090.0,0.2122,0.290411,1
53454,P,22000.0,3563.66,31232.0,0.3125,0.789041,1


In [13]:
df.drop('CallPut',axis=1,inplace=True)
df.rename(columns={'Option Premium': 'Price','Underlying\'s Future Level' : 'Underlying'},inplace=True)

C:\Users\alanw\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\alanw\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
df

,Strike,Price,Underlying,Volatility,Tenor,Put_Flag
0,26000.0,3831.00,22114.0,0.2167,0.627397,0
1,24000.0,8229.00,22080.0,0.2201,0.627397,0
2,24000.0,8209.00,22080.0,0.2198,0.627397,0
3,18000.0,5012.00,22150.0,0.3000,0.627397,1
4,25000.0,5523.00,22150.0,0.2130,0.627397,0
...,...,...,...,...,...,...
53451,31000.0,10976.00,31300.0,0.1855,0.290411,1
53452,28000.0,3408.00,31300.0,0.2233,0.290411,1
53453,28000.0,3292.00,31090.0,0.2122,0.290411,1
53454,22000.0,3563.66,31232.0,0.3125,0.789041,1


In [14]:
from pycaret.regression import *

In [20]:
reg = setup(df,'Price',sample_estimator='',categorical_features = ['Put_Flag'],train_size=0.75,session_id=69)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,69
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(25839, 6)"
4,Missing Values,False
5,Numeric Features,4
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [21]:
compare_models()

IntProgress(value=0, description='Processing: ', max=245)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:18:32
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 8 of 10
Estimator,. . . . . . . . . . . . . . . . . .,Support Vector Machine
ETC,. . . . . . . . . . . . . . . . . .,39.0 Seconds Remaining


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,5604.1725,1.034328e+08,9819.2715,0.2502,1.2022,170.7783
1,Lasso Regression,5604.3054,1.034330e+08,9819.2945,0.2502,1.2023,170.7878
2,Ridge Regression,5604.4625,1.034328e+08,9819.2795,0.2502,1.2023,170.8297
3,Least Angle Regression,5604.1725,1.034328e+08,9819.2715,0.2502,1.2022,170.7783
4,Lasso Least Angle Regression,5622.5570,1.034979e+08,9823.8580,0.2494,1.2076,165.5544
5,Orthogonal Matching Pursuit,5620.1288,1.038412e+08,9842.4293,0.2463,1.2056,150.6001
6,Random Sample Consensus,5305.7054,1.069353e+08,9996.8217,0.2208,1.1367,170.0394
7,Elastic Net,6671.1569,1.217267e+08,10754.0577,0.0896,1.3609,253.5573
8,Bayesian Ridge,7094.9883,1.318603e+08,11233.1181,0.0007,1.4037,278.0383
9,Huber Regressor,6571.2940,1.371824e+08,11449.3666,-0.0376,1.3071,243.0805


KeyboardInterrupt: 